## 1. Key imports

### 1.1. Importing essential libraries

In [6]:
# JSON encoder and decoder
import json
# "Pretty-print" of dict structures
import pprint
# Linear algebra
import numpy as np
from scipy import linalg as la
# Proprietary class and methods allowing to build AHP tree structure
from src.node import Node
from src.utils.file_utils import FileUtils as fu

### 1.2. Importing dataset

In [4]:
# Load model from json file
model_string = open("output/MostSuitableLeader.json").read()
model_dict = json.loads(model_string)

In [13]:
# "Pretty-print" of AHP dict structure
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(model_dict)

{'alternatives': ['Tom', 'Dick', 'Harry'],
 'goal': {'children': [{'children': 'alternatives',
                        'name': 'Experience',
                        'preferences': [[1, 0.25, 4],
                                        [4, 1, 9],
                                        [0.25, 0.111111, 1]]},
                       {'children': 'alternatives',
                        'name': 'Education',
                        'preferences': [[1, 3, 0.2],
                                        [0.333333, 1, 0.142857],
                                        [5, 7, 1]]},
                       {'children': 'alternatives',
                        'name': 'Charisma',
                        'preferences': [[1, 5, 9],
                                        [0.2, 1, 4],
                                        [0.111111, 0.25, 1]]},
                       {'children': 'alternatives',
                        'name': 'Age',
                        'preferences': [[1, 0.333333, 5],
           

In [15]:
# Transformation of dict into tree structure
model = fu.get_node(model_dict["goal"])

## 2. Custom functions for ahp ranking calculation

### 2.1. Eigenvector method

\begin{align}
\mathbf{Aw} & = \begin{pmatrix}
w_{1}/w_{1} & w_{1}/w_{2} & \cdots  & w_{1}/w_{n} \\ 
w_{2}/w_{1} & w_{2}/w_{2} & \cdots  & w_{2}/w_{n} \\ 
\vdots      & \vdots      & \ddots  & \vdots      \\ 
w_{n}/w_{1} & w_{n}/w_{2} & \cdots  & w_{n}/w_{n}
\end{pmatrix} \begin{pmatrix}
w_{1} \\ 
w_{2} \\ 
\vdots \\ 
w_{n}
\end{pmatrix} = \begin{pmatrix}
nw_{1} \\ 
nw_{2} \\ 
\vdots \\ 
nw_{n}
\end{pmatrix} = n\mathbf{w}
\end{align}

\begin{align}
\left\{\begin{matrix}
\mathbf{Aw}=\lambda _{max}\mathbf{w}\\ 
\mathbf{w}^{T}\mathbf{1}=1
\end{matrix}\right.
\end{align}

In [38]:
def getEigRanking(dictionary, verbose = False):
    preferences_array = np.array(dictionary.preferences)
    e_vals, e_vecs = la.eig(preferences_array)
    max_eigv_index = np.argmax(e_vals, axis=0)
    ranking = e_vecs[:,max_eigv_index].real
    ranking = ranking/ranking.sum()
    
    if verbose:
        print(dictionary.name)
        print("Ranking:")
        print(ranking)
    
    if(dictionary.children == "alternatives"):
        return ranking
    else:
        children_ranking = np.array([getEigRanking(child, verbose) for child in dictionary.children])
        return ranking @ children_ranking

### 2.2. Geometric mean method

\begin{align}
w_{i}=(\prod_{j=1}^{n}a_{ij})^{\frac{1}{n}} / \sum_{i=1}^{n}(\prod_{j=1}^{n}a_{ij})^{\frac{1}{n}}
\end{align}

In [44]:
def getGeomRanking(dictionary, verbose = False):
    preferences_array = np.array(dictionary.preferences)
    ranking = np.power(np.prod(preferences_array, axis=1), 1/preferences_array.shape[1])
    ranking = ranking/ranking.sum()
    
    if verbose:
        print(dictionary.name)
        print("Ranking:")
        print(ranking)
    
    if(dictionary.children == "alternatives"):
        return ranking
    else:
        children_ranking = np.array([getGeomRanking(child, verbose) for child in dictionary.children])
        return ranking @ children_ranking

### 2.3. Normalized columns method

In [52]:
def getNormColsRanking(dictionary, verbose = False):
    preferences_array = np.array(dictionary.preferences)
    ranking = np.mean(preferences_array/preferences_array.sum(axis=0)[None,:], axis=1)
    
    if verbose:
        print(dictionary.name)
        print("Ranking:")
        print(ranking)
    
    if(dictionary.children == "alternatives"):
        return ranking
    else:
        children_ranking = np.array([getNormColsRanking(child, verbose) for child in dictionary.children])
        return ranking @ children_ranking

In [39]:
getEigRanking(model, True)

Most Suitable Leader
Ranking:
[ 0.5475693   0.12655527  0.2699499   0.05592553]
Experience
Ranking:
[ 0.21716561  0.71706507  0.06576933]
Education
Ranking:
[ 0.18839409  0.08096118  0.73064473]
Charisma
Ranking:
[ 0.74286665  0.19388163  0.06325172]
Age
Ranking:
[ 0.26543329  0.67162552  0.06294119]


array([ 0.35813676,  0.49278822,  0.14907502])

In [45]:
getGeomRanking(model, True)

Most Suitable Leader
Ranking:
[ 0.54624037  0.12758478  0.26980896  0.0563659 ]
Experience
Ranking:
[ 0.21716561  0.71706506  0.06576933]
Education
Ranking:
[ 0.18839411  0.08096118  0.73064471]
Charisma
Ranking:
[ 0.74286664  0.19388164  0.06325172]
Age
Ranking:
[ 0.26543328  0.67162553  0.06294119]


array([ 0.3580543 ,  0.49218707,  0.14975862])

In [53]:
getNormColsRanking(model, True)

Most Suitable Leader
Ranking:
[ 0.53964382  0.13146424  0.27154789  0.05734404]
Experience
Ranking:
[ 0.21995465  0.71315195  0.0668934 ]
Education
Ranking:
[ 0.19318607  0.08330783  0.7235061 ]
Charisma
Ranking:
[ 0.73518969  0.19941889  0.06539142]
Age
Ranking:
[ 0.26739921  0.66886454  0.06373625]


array([ 0.35906719,  0.48830722,  0.15262559])